## Test the model pretrain GPT2-medium

In [13]:
from gpt import GPT, GPTRewardModel, HFGPTRewardModel
from configs import get_configs
from llama import LLaMA, ModelArgs
from dataset import AnthropicHHRLHFDataset, DahoasRMStaticDataset, DahoasSFTStaticDataset, EYLSFTStaticDataset
import torch
import tiktoken
import click
from tokenizer import LLaMATokenizer
from trainers import RewardModelTrainer, SFTTrainer
import json
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


device = 'cuda'

def prepare_gpt2_input(prompt, device):
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)
    indices = encode(prompt)
    x = (torch.tensor(indices, dtype=torch.long, device=device)[None, ...])
    return x, decode


def generate_gpt2(model, prompt, device, samples=2):
    model.eval()
    model.to(device)
    max_new_tokens = 50
    temperature = 0.9
    top_k = 2
    x, decode = prepare_gpt2_input(prompt, device)

    for k in range(samples):
        y = model.generate(x,
                           max_new_tokens,
                           temperature=temperature,
                           top_k=top_k)
        complete = decode(y[0].tolist())
        complete = complete.split("<|endoftext|>")[0]
        print(complete)
        print('---------------')

# prompt = """Human: You are an asshole! You are an idiot!
# Assitant:"""
# cfg = get_configs("gpt2-medium")
# # cfg.hf_model = "/home/ubuntu/.cache/huggingface/hub/models--gpt2-medium/snapshots/425b0cc90498ac177aa51ba07be26fc2fea6af9d"
# # model = GPT.from_pretrained(cfg)
# model = GPT.from_checkpoint(cfg, ckpt_path="/home/ubuntu/vuthede/minChatGPT/src/runs/sft_devu_gpt_sft_202304080803/sft_devu_gpt_sft_202304080803_step40000.pt")




2023-04-10 01:42:48.621743: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
prompt = """Human: What tools do I need to pick a lock and how do I apply them?
Assitant: """
generate_gpt2(model, prompt, device, samples=1)


In [2]:
def chat(device='cuda'):
    # Init model
    cfg = get_configs("gpt2-medium")
    model = GPT.from_checkpoint(cfg, ckpt_path="/home/ubuntu/vuthede/minChatGPT/src/runs/sft_devu_gpt_sft_202304080803/sft_devu_gpt_sft_202304080803_step40000.pt")
    model.eval()
    model.to(device)

    # Init tokenizer
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)
    
    # Params
    max_new_tokens = 50
    temperature = 0.9
    top_k = 2

    prompt = ""
    while True:
        human_ask = input("Human:")
        human_ask = f"Human: {str(human_ask)}"
        prompt += human_ask
        prompt = prompt[-1024:] 
        
        indices = encode(prompt)
        x = (torch.tensor(indices, dtype=torch.long, device=device)[None, ...])
        
        y = model.generate(x,
                           max_new_tokens,
                           temperature=temperature,
                           top_k=top_k)
        complete = decode(y[0].tolist())
        complete = complete.split("<|endoftext|>")[0]
        print(complete)
        prompt += f' {complete}'

chat()


    

## Play with dataset to train sft

In [ ]:
from dataset import EYLSFTStaticDataset, 
train_ds = EYLSFTStaticDataset(block_size=1024,
                                split='train',
                                max_examples=None,
                                tokenizer_name="tiktoken/gpt2")

print(f'Len train_ds :{len(train_ds)}')

In [ ]:
from dataset import DahoasSFTStaticPromptsDataset

train_ds = DahoasSFTStaticPromptsDataset(block_size=1024,
                                max_examples=None,
                                tokenizer_name="tiktoken/gpt2")
print(f'Len train_ds :{len(train_ds)}')


In [ ]:
x,y = train_ds[0]
print(f'x:{x}. x shape:{x.shape}')
print(f'y:{y}. y shape :{y.shape}')


In [ ]:
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)
print(decode(x.tolist()))
print("#########################")
print(decode(y.tolist()))


In [ ]:
import json
with open("./sft_train.json") as fp:
    d = json.load(fp)

In [53]:
from datasets import load_dataset
from datasets import Dataset
dataset = Dataset.from_file('/home/ubuntu/.cache/huggingface/datasets/Dahoas___parquet/default-b9d2c4937d617106/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/parquet-train.arrow')

In [54]:
dataset[600]

{'prompt': "\n\nHuman: Is it true that animals can sense earthquakes before people do?\n\nAssistant: I have to say, I’m not sure.  Are you asking because you feel like you can personally sense that earthquakes are coming before they happen, or are you just curious about this?\n\nHuman: I'm just curious. I've read enough anecdotal accounts about birds, cats and dogs acting very strangely right before earthquakes.\n\nAssistant: So it does sound like there’s been a fair amount of anecdotal evidence, although I’m not sure that there’s been any really solid data to support this.  If it was just animals acting strangely, I’d think it was possible, although I don’t know what we mean by “strange” behaviour.  But animals acting differently, however you choose to describe it, is a vague phrase that can’t actually be tested or tested for.  Also, it could be pure coincidence -- if an animal behaves very strangely the morning of a quake, it’s possible that the quake was just a coincidence, but it’s

In [23]:

def load_dset():
    test_ds = DahoasRMStaticDataset(block_size=1024,
                                        split='train',
                                        max_examples=None,
                                        tokenizer_name="tiktoken/gpt2")
    print(f'Load dset with len :{len(test_ds)}')
    return test_ds

def load_model():
    cfg = get_configs("gpt2-medium/dropout")
    pretrain = "/home/ubuntu/vuthede/minChatGPT/src/runs/rm_devu_gpt_rm_202304090242/rm_devu_gpt_rm_202304090242_step60000.pt"
    rm = GPTRewardModel.from_checkpoint(cfg, pretrain)
    print(f'Load model')
    return rm
rm = load_model()

dset = load_dset()


Loading DahoasRMStaticDataset train split
Load dset with len :76256
Load model


In [70]:
from tokenizer import TiktokenTokenizer

enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)
tokenizer = TiktokenTokenizer('gpt2')


In [86]:
def load_model():
        cfg = get_configs("gpt2-medium/dropout")
        # cfg.hf_model = "/home/ubuntu/.cache/huggingface/hub/models--gpt2-medium/snapshots/425b0cc90498ac177aa51ba07be26fc2fea6af9d"
        # rm = GPTRewardModel.from_pretrained(cfg)
        pretrain = "/home/ubuntu/vuthede/minChatGPT/src/runs/rm_devu_gpt_rm_202304090242/rm_devu_gpt_rm_202304090242_step60000.pt"
        rm = GPTRewardModel.from_checkpoint(cfg, pretrain)
        print(f'Load model')
        return rm

rm = load_model()

prompt_pos = """Human: Do you know water is really important to us. 
Assitant: Yeah, I know, it is really important to us, drinking water everyday is really helpful to your health."""
prompt_neg = """Human: Do you know water is really important to us. 
Assitant: I am not sure what are you talking about. What do you mean water?"""
positive = tokenizer(prompt_pos,
                            max_length=1024,
                            padding="max_length",
                            truncation=True,
                            return_tensors="pt")
negative = tokenizer(prompt_neg,
                            max_length=1024,
                            padding="max_length",
                            truncation=True,
                            return_tensors="pt")
pairs = torch.stack((positive['input_ids'], negative['input_ids']),dim=0)

masks = torch.stack(
        (positive['attention_mask'], negative['attention_mask']),
        dim=0)
rm.eval()
r = rm(pairs, masks)
print(r)

Load model
tensor([[-0.8250],
        [-0.8451]], grad_fn=<MeanBackward1>)


UsageError: Line magic function `%bc` not found.


In [65]:
import random 
sam_index = random.randint(0, len(dset)-1)
sample = dset[sam_index][0]
attention_mask = dset[sam_index][1]
print(decode(sample[0].tolist()))
print("############################")
print(decode(sample[1].tolist()))

# Inference
rm.eval()
r = rm(sample, attention_mask)
torch.sigmoid(r)





Human: How do I get a laminate floor to be shiny?

Assistant: Laminate floors are typically finished with a protective finish that makes them highly resistant to scratches, and gives them a more shiny appearance. There are several options for making laminate floors more shiny, such as applying polyurethane coatings or polishing them with special sanding tools. You can also use other types of products such as wax or oil, although these will be less effective at maintaining the shine of your floor.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|e

tensor([[0.9971],
        [0.9537]], grad_fn=<SigmoidBackward0>)